# Procesamiento de Lenguaje Natural con LLMs
### Modelo Clásico a Modelos Modernos 🚀

Este notebook muestra la evolución del NLP:
1. Fundamentos clásicos (preprocesamiento y representación)
2. Clasificación tradicional
3. LLMs modernos

## PARTE 1: FUNDAMENTOS CLÁSICOS DE NLP

In [1]:
!pip install nltk transformers torch scikit-learn -q

In [2]:
import nltk
import re
from collections import Counter
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)
nltk.download('stopwords', quiet=True)

True

In [3]:
# Dataset simple para clasificación de sentimientos
reviews = [
    "Me encanta este producto, es increíble!",
    "Pésima experiencia, no lo recomiendo",
    "Es aceptable, cumple lo esperado",
    "Excelente compra, superó expectativas",
    "Horrible servicio, muy decepcionado",
    "Hay productos peores, pero por su precio vale la pena"
]
sentimientos = ['positivo', 'negativo', 'neutral', 'positivo', 'negativo', 'neutral']

print("=== DATASET ORIGINAL ===")
for i, (review, sent) in enumerate(zip(reviews, sentimientos), 1):
    print(f"{i}. [{sent}] {review}")

=== DATASET ORIGINAL ===
1. [positivo] Me encanta este producto, es increíble!
2. [negativo] Pésima experiencia, no lo recomiendo
3. [neutral] Es aceptable, cumple lo esperado
4. [positivo] Excelente compra, superó expectativas
5. [negativo] Horrible servicio, muy decepcionado
6. [neutral] Hay productos peores, pero por su precio vale la pena


### Preprocesamiento Clásico

**PREPROCESAMIENTO**: Limpiar y normalizar texto para análisis.
- Tokenización: dividir en palabras
- Stop words: remover palabras comunes sin significado
- Stemming: reducir palabras a su raíz

In [4]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

stop_words = set(stopwords.words('spanish'))
stemmer = SnowballStemmer('spanish')

def preprocesar(texto):
    # Limpiar y tokenizar
    texto = re.sub(r'[^\w\s]', '', texto.lower())
    tokens = word_tokenize(texto)
    # Remover stop words y hacer stemming
    tokens = [stemmer.stem(t) for t in tokens if t not in stop_words]
    return ' '.join(tokens)

reviews_procesados = [preprocesar(r) for r in reviews]

print("\n=== DESPUÉS DEL PREPROCESAMIENTO ===")
for i, review in enumerate(reviews_procesados, 1):
    print(f"{i}. {review}")


=== DESPUÉS DEL PREPROCESAMIENTO ===
1. encant product increibl
2. pesim experient recom
3. acept cumpl esper
4. excelent compr super expect
5. horribl servici decepcion
6. product peor preci val pen


### Representación: Bag of Words

**BAG OF WORDS**: Representa texto como vector de frecuencias de palabras.
Cada palabra del vocabulario es una dimensión.
Problema: Pierde contexto y semántica.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(reviews_procesados)

print("\n=== BAG OF WORDS (TF-IDF) ===")
print(f"Vocabulario: {vectorizer.get_feature_names_out()}")
print(f"Dimensión: {X.shape}")


=== BAG OF WORDS (TF-IDF) ===
Vocabulario: ['acept' 'compr' 'cumpl' 'decepcion' 'encant' 'esper' 'excelent' 'expect'
 'experient' 'horribl' 'increibl' 'pen' 'peor' 'pesim' 'preci' 'product'
 'recom' 'servici' 'super' 'val']
Dimensión: (6, 20)


### Clasificación Tradicional

**CLASIFICACIÓN CLÁSICA**: Usamos algoritmos como Naive Bayes o SVM.
Requieren mucho preprocesamiento y features engineering.

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score

clf = MultinomialNB()
scores = cross_val_score(clf, X, sentimientos, cv=2)
print(f"\n=== CLASIFICACIÓN TRADICIONAL (Naive Bayes) ===")
print(f"Accuracy promedio: {scores.mean():.2f}")


=== CLASIFICACIÓN TRADICIONAL (Naive Bayes) ===
Accuracy promedio: 0.33


## PARTE 2: ¡LA REVOLUCIÓN DE LOS LLMs! 🤖✨

**LLMs (Large Language Models)**: Modelos masivos entrenados en billones de palabras.
- Entienden contexto, semántica y relaciones complejas
- No necesitan preprocesamiento manual
- Pueden hacer múltiples tareas sin reentrenamiento (zero-shot)
- Ejemplos: BERT, GPT, LLaMA, etc.


In [ ]:
from transformers import pipeline

print("\n" + "="*70)
print("🚀 CARGANDO MODELO DE LENGUAJE MODERNO...")
print("="*70)

# Cargamos un modelo preentrenado en español para análisis de sentimientos
# Este modelo ya fue entrenado en millones de textos
clasificador_llm = pipeline(
    "sentiment-analysis",
    model="pysentimiento/robertuito-sentiment-analysis"
)

print("✅ Modelo cargado: RoBERTuito (BERT en español)")
print("   Entrenado en millones de tweets en español\n")


🚀 CARGANDO MODELO DE LENGUAJE MODERNO...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/925 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/435M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

Device set to use cpu


✅ Modelo cargado: RoBERTuito (BERT en español)
   Entrenado en millones de tweets en español



### Análisis con LLM - Zero Shot

**ZERO-SHOT**: El modelo puede clasificar SIN ver ejemplos de entrenamiento.
Ya aprendió sobre sentimientos de su entrenamiento masivo.

In [ ]:
print("=== CLASIFICACIÓN CON LLM (Sin preprocesamiento) ===\n")

for i, review in enumerate(reviews, 1):
    resultado = clasificador_llm(review)[0]
    etiqueta = resultado['label']
    confianza = resultado['score']

    # Mapeo de etiquetas
    etiqueta_es = {'POS': '😊 Positivo', 'NEG': '😞 Negativo', 'NEU': '😐 Neutral'}

    print(f"{i}. \"{review}\"")
    print(f"   Predicción: {etiqueta_es.get(etiqueta, etiqueta)} (confianza: {confianza:.2%})")
    print()

=== CLASIFICACIÓN CON LLM (Sin preprocesamiento) ===

1. "Me encanta este producto, es increíble!"
   Predicción: 😊 Positivo (confianza: 97.81%)

2. "Pésima experiencia, no lo recomiendo"
   Predicción: 😞 Negativo (confianza: 95.58%)

3. "Es aceptable, cumple lo esperado"
   Predicción: 😊 Positivo (confianza: 89.51%)

4. "Excelente compra, superó expectativas"
   Predicción: 😊 Positivo (confianza: 95.00%)

5. "Horrible servicio, muy decepcionado"
   Predicción: 😞 Negativo (confianza: 97.84%)



### Casos más complejos

In [ ]:
print("\n" + "="*70)
print("🧪 PROBANDO CON CASOS DESAFIANTES")
print("="*70 + "\n")

casos_complejos = [
    "No es que sea malo, pero definitivamente no es bueno",  # Negación compleja
    "Esperaba más, aunque tiene sus puntos positivos",       # Sentimiento mixto
    "¡Qué desastre tan espectacular! (Es ironía)",          # Ironía
    "Increíble cómo algo tan caro puede ser tan mediocre"   # Sarcasmo
]

for caso in casos_complejos:
    resultado = clasificador_llm(caso)[0]
    etiqueta = {'POS': '😊', 'NEG': '😞', 'NEU': '😐'}.get(resultado['label'], '🤔')

    print(f"📝 \"{caso}\"")
    print(f"   {etiqueta} {resultado['label']} ({resultado['score']:.1%})\n")


🧪 PROBANDO CON CASOS DESAFIANTES

📝 "No es que sea malo, pero definitivamente no es bueno"
   😞 NEG (82.8%)

📝 "Esperaba más, aunque tiene sus puntos positivos"
   😐 NEU (67.1%)

📝 "¡Qué desastre tan espectacular! (Es ironía)"
   😞 NEG (77.9%)

📝 "Increíble cómo algo tan caro puede ser tan mediocre"
   😞 NEG (97.2%)



## PARTE 3: GENERACIÓN DE TEXTO CON LLMs 🎨

Los LLMs también pueden **GENERAR** texto coherente.
Vamos a usar un modelo generativo en español.

In [ ]:
print("\n" + "="*70)
print("🎨 GENERACIÓN DE TEXTO")
print("="*70 + "\n")

# Modelo generativo (más pequeño para Google Colab)
generador = pipeline(
    "text-generation",
    model="DeepESP/gpt2-spanish",
    max_length=80
)

prompts = [
    "La inteligencia artificial es",
    "En el futuro, los robots",
    "El curso de IA me parece"
]

for prompt in prompts:
    texto_generado = generador(prompt, do_sample=True, temperature=0.7)[0]['generated_text']
    print(f"💭 Prompt: \"{prompt}\"")
    print(f"✨ Generado: {texto_generado}\n")


🎨 GENERACIÓN DE TEXTO



config.json:   0%|          | 0.00/914 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/261M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/262 [00:00<?, ?B/s]

Device set to use cpu


💭 Prompt: "La inteligencia artificial es"
✨ Generado: La inteligencia artificial es un factor que, por razones que no se pueden explicar, es esencial para la supervivencia. Un ejemplo más de la inteligencia artificial es el de la inteligencia artificial. Si se trata de una inteligencia artificial, resulta esencial para la supervivencia de

💭 Prompt: "En el futuro, los robots"
✨ Generado: En el futuro, los robots, los ordenadores y los ordenadores se convertirán en una especie de robot y en un robot. 

—Así que han venido a la Tierra —dijo un robot—. Y eso significa que la Corporación no tiene necesidad de

💭 Prompt: "El curso de IA me parece"
✨ Generado: El curso de IA me parece de lo más adecuado. 

—No es mi trabajo —digo. 

—Es tu trabajo, pero no te preocupes por él. 

—Supongo que tendré que dejarte. 

Niega con la



## PARTE 4: COMPARACIÓN Y CONCLUSIONES

In [ ]:
print("\n" + "="*70)
print("📊 MÉTODOS CLÁSICOS vs LLMs")
print("="*70 + "\n")

comparacion = """
┌─────────────────────────┬──────────────────────┬─────────────────────┐
│ ASPECTO                 │ MÉTODOS CLÁSICOS     │ LLMs                │
├─────────────────────────┼──────────────────────┼─────────────────────┤
│ Preprocesamiento        │ Extensivo y manual   │ Mínimo o ninguno    │
│ Features                │ Manual (BoW, TF-IDF) │ Automáticas         │
│ Contexto                │ Limitado             │ Excelente           │
│ Semántica               │ Básica               │ Profunda            │
│ Datos necesarios        │ Miles por clase      │ Zero/Few-shot       │
│ Velocidad inferencia    │ Muy rápida           │ Más lenta           │
│ Recursos computación    │ Bajos                │ Altos               │
│ Tareas múltiples        │ Un modelo por tarea  │ Modelo multiuso     │
└─────────────────────────┴──────────────────────┴─────────────────────┘
"""

print(comparacion)


📊 MÉTODOS CLÁSICOS vs LLMs


┌─────────────────────────┬──────────────────────┬─────────────────────┐
│ ASPECTO                 │ MÉTODOS CLÁSICOS     │ LLMs                │
├─────────────────────────┼──────────────────────┼─────────────────────┤
│ Preprocesamiento        │ Extensivo y manual   │ Mínimo o ninguno    │
│ Features                │ Manual (BoW, TF-IDF) │ Automáticas         │
│ Contexto                │ Limitado             │ Excelente           │
│ Semántica               │ Básica               │ Profunda            │
│ Datos necesarios        │ Miles por clase      │ Zero/Few-shot       │
│ Velocidad inferencia    │ Muy rápida           │ Más lenta           │
│ Recursos computación    │ Bajos                │ Altos               │
│ Tareas múltiples        │ Un modelo por tarea  │ Modelo multiuso     │
└─────────────────────────┴──────────────────────┴─────────────────────┘



## BONUS: Clasificación Personalizada con Few-Shot

**FEW-SHOT LEARNING**: Damos al LLM solo unos pocos ejemplos y aprende la tarea.
¡No necesita reentrenamiento completo!

In [ ]:
print("\n" + "="*70)
print("🎯 BONUS: FEW-SHOT LEARNING")
print("="*70 + "\n")

# Usamos un modelo más potente para few-shot
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

modelo_nombre = "dccuchile/bert-base-spanish-wwm-cased"
tokenizer = AutoTokenizer.from_pretrained(modelo_nombre)
modelo = AutoModelForSequenceClassification.from_pretrained(
    modelo_nombre,
    num_labels=3
)

print("💡 Con few-shot, el modelo aprende de pocos ejemplos y generaliza.")
print("   Útil cuando tienes datos limitados o tareas muy específicas.\n")


🎯 BONUS: FEW-SHOT LEARNING



tokenizer_config.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


💡 Con few-shot, el modelo aprende de pocos ejemplos y generaliza.
   Útil cuando tienes datos limitados o tareas muy específicas.



## CONCLUSIONES Y RECURSOS

In [ ]:
print("\n" + "="*70)
print("🎓 CONCLUSIONES")
print("="*70 + "\n")

print("""
🔗 RECURSOS:
   • HuggingFace Models: https://huggingface.co/models
   • Transformers Library: https://huggingface.co/docs/transformers
   • Papers With Code (NLP): https://paperswithcode.com/area/natural-language-processing
""")

print("\n✨ ¡El futuro del NLP es emocionante! Sigue explorando... ✨\n")


🎓 CONCLUSIONES


🔗 RECURSOS:
   • HuggingFace Models: https://huggingface.co/models
   • Transformers Library: https://huggingface.co/docs/transformers
   • Papers With Code (NLP): https://paperswithcode.com/area/natural-language-processing


✨ ¡El futuro del NLP es emocionante! Sigue explorando... ✨

